# Load the datasets

In [1]:
import pandas as pd
from sklearn.impute import SimpleImputer


In [2]:
# Load datasets

ieee_identity = pd.read_csv(
    "../datasets/ieee-fraud-detection-datasets/train/train_identity.csv"
)

ieee_transaction = pd.read_csv(
    "../datasets/ieee-fraud-detection-datasets/train/train_transaction.csv"
)

# Merge the datasets on 'TransactionID'
merged_df = pd.merge(ieee_transaction, ieee_identity, on="TransactionID", how="left")

# Check the shape and head of the merged dataset
print("Merged shape:", merged_df.shape)
print(merged_df.head())


Merged shape: (590540, 434)
   TransactionID  isFraud  TransactionDT  TransactionAmt ProductCD  card1  \
0        2987000        0          86400            68.5         W  13926   
1        2987001        0          86401            29.0         W   2755   
2        2987002        0          86469            59.0         W   4663   
3        2987003        0          86499            50.0         W  18132   
4        2987004        0          86506            50.0         H   4497   

   card2  card3       card4  card5  ...                id_31  id_32  \
0    NaN  150.0    discover  142.0  ...                  NaN    NaN   
1  404.0  150.0  mastercard  102.0  ...                  NaN    NaN   
2  490.0  150.0        visa  166.0  ...                  NaN    NaN   
3  567.0  150.0  mastercard  117.0  ...                  NaN    NaN   
4  514.0  150.0  mastercard  102.0  ...  samsung browser 6.2   32.0   

       id_33           id_34  id_35 id_36 id_37  id_38  DeviceType  \
0        NaN

In [3]:
merged_df.to_csv("../datasets/ieee-processed/ieee-train-merged.csv")


Snippet from https://dl.acm.org/doi/10.1145/3677052.3698692 on how the dataset was pre-pre-processed.

<img src="./resources/grab-Retrieval Augmented Fraud Detection_Paper_Preprocessing.png" width="40%">

## Processing the data according to the paper

#### Features with over 20% missing values were removed


In [4]:
# Calculate missing percentages
missing_percent = merged_df.isnull().mean() * 100

missing_df = pd.DataFrame({
    'Column Name': missing_percent.index,
    'Missing Percentage (%)': missing_percent.values
}).sort_values(by='Missing Percentage (%)', ascending=False)

# Display the missing percentages neatly
print("\n=== Missing Values Percentage per Column ===\n")
print(missing_df.round(2).to_string(index=False))



=== Missing Values Percentage per Column ===

   Column Name  Missing Percentage (%)
         id_24                   99.20
         id_25                   99.13
         id_07                   99.13
         id_08                   99.13
         id_21                   99.13
         id_26                   99.13
         id_27                   99.12
         id_23                   99.12
         id_22                   99.12
         dist2                   93.63
            D7                   93.41
         id_18                   92.36
           D13                   89.51
           D14                   89.47
           D12                   89.04
         id_03                   88.77
         id_04                   88.77
            D6                   87.61
         id_33                   87.59
         id_10                   87.31
         id_09                   87.31
            D9                   87.31
            D8                   87.31
         id_30   

In [5]:
# Columns exceeding 25%
cols_to_drop_df = missing_df[missing_df['Missing Percentage (%)'] > 25]

if not cols_to_drop_df.empty:
    print("\n=== Columns to be Dropped (Missing > 25%) ===\n")
    print(cols_to_drop_df.round(2).to_string(index=False))
    print(f"\n🚨 Removing {len(cols_to_drop_df)} columns due to high missing values (>25%).")
else:
    print("\n✅ No columns exceed 25% missing values. Nothing will be dropped.")



=== Columns to be Dropped (Missing > 25%) ===

  Column Name  Missing Percentage (%)
        id_24                   99.20
        id_25                   99.13
        id_07                   99.13
        id_08                   99.13
        id_21                   99.13
        id_26                   99.13
        id_27                   99.12
        id_23                   99.12
        id_22                   99.12
        dist2                   93.63
           D7                   93.41
        id_18                   92.36
          D13                   89.51
          D14                   89.47
          D12                   89.04
        id_03                   88.77
        id_04                   88.77
           D6                   87.61
        id_33                   87.59
        id_10                   87.31
        id_09                   87.31
           D9                   87.31
           D8                   87.31
        id_30                   86.87
  

In [6]:
cols_to_drop = []

for col in cols_to_drop_df['Column Name']:
    cols_to_drop.append(col)

print(cols_to_drop), len(cols_to_drop)


['id_24', 'id_25', 'id_07', 'id_08', 'id_21', 'id_26', 'id_27', 'id_23', 'id_22', 'dist2', 'D7', 'id_18', 'D13', 'D14', 'D12', 'id_03', 'id_04', 'D6', 'id_33', 'id_10', 'id_09', 'D9', 'D8', 'id_30', 'id_32', 'id_34', 'id_14', 'V142', 'V158', 'V140', 'V162', 'V141', 'V161', 'V157', 'V146', 'V156', 'V155', 'V154', 'V153', 'V149', 'V147', 'V148', 'V163', 'V139', 'V138', 'V160', 'V151', 'V152', 'V145', 'V144', 'V143', 'V159', 'V164', 'V165', 'V166', 'V150', 'V337', 'V333', 'V336', 'V335', 'V334', 'V338', 'V339', 'V324', 'V332', 'V325', 'V330', 'V329', 'V328', 'V327', 'V326', 'V322', 'V323', 'V331', 'DeviceInfo', 'id_13', 'id_16', 'V278', 'V277', 'V252', 'V253', 'V254', 'V257', 'V258', 'V242', 'V261', 'V262', 'V263', 'V264', 'V249', 'V266', 'V267', 'V268', 'V269', 'V273', 'V274', 'V275', 'V276', 'V265', 'V260', 'V247', 'V246', 'V240', 'V237', 'V236', 'V235', 'V233', 'V232', 'V231', 'V230', 'V229', 'V228', 'V226', 'V225', 'V224', 'V223', 'V219', 'V218', 'V217', 'V243', 'V244', 'V248', 'V241'

(None, 252)

We have to drop `252` columns (names mentioned above) as they have more than 25% of it's values missing.

In [7]:
# Drop these columns from the dataset
merged_df = merged_df.drop(columns=cols_to_drop)

# Print the updated dataset shape
print(f"New dataset shape after dropping columns: {merged_df.shape}")


New dataset shape after dropping columns: (590540, 182)


#### Step 2: Identify Numerical & Categorical Columns & Impute Missing Values


In [8]:

num_cols = merged_df.select_dtypes(include=['int64', 'int32', 'float64', 'float32']).columns
cat_cols = merged_df.select_dtypes(include=['object', 'category']).columns

# Print the counts
print(f"Identified {len(num_cols)} numerical columns and {len(cat_cols)} categorical columns. Shape of the dataset is {merged_df.shape}.")

# Validation Check: Ensure num_cols + cat_cols equals the total number of columns
assert len(num_cols) + len(cat_cols) == merged_df.shape[1], "Column count mismatch! Check data types."



Identified 178 numerical columns and 4 categorical columns. Shape of the dataset is (590540, 182).


In [9]:
print("\nStarting missing value imputation...")

# Impute numerical columns with median
num_imputer = SimpleImputer(strategy='median')
merged_df[num_cols] = num_imputer.fit_transform(merged_df[num_cols])
print("Numerical missing values filled using median.")

# Impute categorical columns with mode (most frequent value)
cat_imputer = SimpleImputer(strategy='most_frequent')
merged_df[cat_cols] = cat_imputer.fit_transform(merged_df[cat_cols])
print("Categorical missing values filled using mode (most frequent value).")



Starting missing value imputation...
Numerical missing values filled using median.
Categorical missing values filled using mode (most frequent value).


In [10]:
### Step 4: Final Checks ###
print("\nFinal validation checks:")

# Check if any missing values remain
remaining_missing_values = merged_df.isnull().sum().sum()
print(f"Total missing values remaining: {remaining_missing_values}")  # Should be 0
assert remaining_missing_values == 0, "Imputation failed, missing values still exist!"

# Print the final dataset shape
print(f"Final dataset shape: {merged_df.shape}")

print("\nDataset cleaning complete! ✅ The dataset is now ready for further processing.")



Final validation checks:
Total missing values remaining: 0
Final dataset shape: (590540, 182)

Dataset cleaning complete! ✅ The dataset is now ready for further processing.


In [11]:
merged_df.to_csv("../datasets/ieee-processed/ieee-train-merged_imputed_cleaned.csv", index=False)


### Debugging Code

In [12]:
# merged_df.shape


In [13]:
# merged_df.columns.tolist()


In [14]:
# merged_df_readback = pd.read_csv("../datasets/ieee-processed/ieee-train-merged_imputed_cleaned.csv", index_col = False)
# merged_df_readback.columns
